# Diffusion Simulator/Animator

## Diffusion equation:
### The continuity equation:
$$\nabla \cdot \vec{j} + \frac{\partial \phi}{\partial t} = 0$$

$\phi$ : Density of diffusing quantity

$\vec{j}$ : Current density

### Fick's first law:
$$\vec{j} = - \overleftrightarrow{D}[\phi,\vec{r}] \nabla\phi[\vec{r},t]$$
$\overleftrightarrow{D}[\phi,\vec{r}]$ : Diffusion (tensor) coefficient for density $\phi$ at position $\vec{r}$

Combining the above equations gives the general diffusion equation:
$$\frac{\partial\phi}{\partial t} = \nabla \Big( \overleftrightarrow{D}[\phi,\vec{r}] \nabla\phi \Big) = \partial^{\alpha} \big(D_{\alpha\beta}\partial^{\beta} \phi \big)$$ 

### The typical diffusion:
When D is an scalar constant(isotropic and independent of density and position), the diffusion equation reduces to:
$$\frac{\partial\phi}{\partial t} = D \nabla^2 \phi$$ 

## Current Implementation:
$$\overleftrightarrow{D}[\phi,\vec{r}] \equiv D[\vec{r}]\bigg(1-\frac{B[\vec{r}]}{|\nabla \phi|}\bigg) $$ 
The simulation here evolves the diffusing quantity given:
1. The boundary conditions,
2. The initial distribution,
3. The diffusion coefficient, $D[\vec{r}]$, as function of position,
4. Minimum gradient, $B[\vec{r}]$, required to drive current,




---
## Setup matplotlib backend

In [1]:
## embedded interactive plot (e.g.: jupyter notebook)
# %matplotlib notebook
## embedded interactive plot (e.g.: jupyter notebook)
## spawn a separate window (e.g.: jupyter lab)
%matplotlib qt
## spawn a separate window (e.g.: jupyter lab)

---
## Import everything required

In [2]:
## src.diffusion is dependent on the path
import src.diffusion as dfsn
## src.diffusion is dependent on the path
import numpy as np

---
## Diffusion for temperature
This simulates the heat diffusion with constant boundary temperature.

1. Instantiate the simulator

In [3]:
temperature = dfsn.Simulator(n=(10,10), bc='open')
# n : shape of lattice grid
# dL : lattice grid spacing
# bc : is boundary condition
 # 'open'/1 : edges can have current (boundary values are fixed)

2. Initialize the Diffusion coefficent, Minimum gradient to drive current, and temperatures at the boundaries and the bulk.

In [4]:
f_start = 0
f_at_boundary = [0, 0, 0, np.linspace(0,1,10)]
temperature.initialize(f=f_start, f_at_b=f_at_boundary, D=1, B=0)
# f : the initial distribution of temperature
# f_at_b : the fixed boundary temperatures
# D : the Diffusion coefficient
# B : the Minimum gradient required to drive current.

3. Evolve for finite duration

In [5]:
temperature.evolve(Dt=1,dt=0.01)
# Dt : duration of evolution
# dt : time step for evolution, this may be reduced to avoid instabilities
  # ddt : the time-step subdivision.

Simulating (ddt=0.010000) : t=0.000000-->1.000000 - Evolved!


4. Evolve until steady state reached

In [6]:
temperature.steadyState(precision=0.01)
# precision : change in temperature reduces below this value
  # ddt : time-step used for evolution.

Simulating (ddt=0.125000): t=1.000000-->1.750000 - Steady State reached!


---
## Diffusion for particle density
This simulates the diffusiion of particle density in a region where there is no flux in or out of the material.

1. Instantiate the simulator

In [7]:
rho = dfsn.Simulator(n=(100,), dL=1, bc=0)
# n : shape of lattice grid
# dL : lattice grid spacing
# bc : is boundary condition
  # 'closed'/0 : no current at the edges

2. Initialize the Diffusion coefficent, Minimum gradient to drive current, and temperatures at the boundaries and the bulk.

In [8]:
f_start = np.concatenate([np.zeros(50),np.ones(50)])
rho.initialize(f=f_start, D=1, B=0)
# f : the initial distribution of rho
# D : the Diffusion coefficient
# B : the Minimum gradient required to drive current.

3. Evolve for finite duration

In [9]:
rho.evolve(Dt=10,dt=0.1)
# Dt : duration of evolution
# dt : time step for evolution, this may be reduced to avoid instabilities
  # ddt : the time-step subdivision.

Simulating (ddt=0.100000) : t=0.000000-->10.000000 - Evolved!


4. Evolve until steady state reached

In [10]:
rho.steadyState(precision=0.0001)
# precision : change in rho reduces below this value
  # ddt : time-step used for evolution.

Simulating (ddt=0.250000): t=10.000000-->580.000000 - Steady State reached!


---
## Checkpoint

In [11]:
temperature.save_checkpoint(filename="data/temp_chkpt")
# filename: with path without extension
  # 2 files generated filename+".json" and filename+".npy"

Saved to: data/temp_chkpt.json & data/temp_chkpt.npy


In [12]:
rho.save_checkpoint(filename="data/rho_chkpt")
# filename: with path without extension
  # 2 files generated filename+".json" and filename+".npy"

Saved to: data/rho_chkpt.json & data/rho_chkpt.npy


### Load Checkpoint

In [13]:
tempck = dfsn.Simulator()
tempck.load_checkpoint("data/temp_chkpt")
# filename: with path without extension
  # Look for 2 files - filename+".json" and filename+".npy"

Loaded: data/temp_chkpt.json & data/temp_chkpt.npy


In [14]:
rhock = dfsn.Simulator()
rhock.load_checkpoint("data/rho_chkpt")
# filename: with path without extension
  # Look for 2 files - filename+".json" and filename+".npy"

Loaded: data/rho_chkpt.json & data/rho_chkpt.npy


### Check integrity

In [15]:
tempckdata = vars(tempck)
tempdata = vars(temperature)
for key in ["dL","n","t","bc","f","D","B"]:
    if np.all(tempckdata[key]==tempdata[key]):
        print(f"{key}: OK")
    else:
        print(f"{key}: Nope")

dL: OK
n: OK
t: OK
bc: OK
f: OK
D: OK
B: OK


In [16]:
rhockdata = vars(rhock)
rhodata = vars(rho)
for key in ["dL","n","t","bc","f","D","B"]:
    if np.all(rhockdata[key]==rhodata[key]):
        print(f"{key}: OK")
    else:
        print(f"{key}: Nope")

dL: OK
n: OK
t: OK
bc: OK
f: OK
D: OK
B: OK


---
## Animated 1D diffusion

### Evolve for finite duration

In [39]:
rho = dfsn.Simulator(n=(100,), bc=0)
f_start = np.concatenate([np.zeros(50),np.ones(50)])
rho.initialize(f=f_start, D=1, B=0)
# Animator must be called with a Simulator object
# takes arguments simObj, savefile, figsizeX, figsizeY, display
# simObj must be an object of Simulator Class
with dfsn.Animator(simObj=rho, savefile="data/rhoEvo1D.mp4", force_initial_size=False) as plotter:
    plotter.start_animation(simulate="evolve", kwargs={'Dt':400,'dt':0.01, 'plot_realtime_interval':0.1}, display=True)
    # Do as many 
    # start_animation takes simulate "<function_name>" of Simulator class
    # kwargs takes in a dictionary of arguments to be passed to the "evolve" function
    # here # kwargs = {'Dt':400,'dt':0.01, plot_realtime_interval=0.1} updates plot every 0.1seconds (default)

Simulating (ddt=0.010000) : t=0.000000-->400.000000 - Evolved!
Animation saved to: data/rhoEvo1D.mp4


### Evolve to steady state

In [36]:
rho = dfsn.Simulator(n=(100,), bc=0)
f_start = np.concatenate([np.zeros(50),np.ones(50)])
rho.initialize(f=f_start, D=1, B=0)
# Animator must be called with a Simulator object
# takes arguments simObj, savefile, figsizeX, figsizeY, display
# simObj must be an object of Simulator Class
with dfsn.Animator(simObj=rho, savefile="data/rhoSS1D.mp4") as plotter:
    plotter.start_animation(simulate="steadyState", kwargs={'precision':0.001, 'plot_dt_steps':100})
    # start_animation takes simulate "<function_name>" of Simulator class
    # kwargs takes in a dictionary of arguments to be passed to the "steadyState" function
    # here # kwargs = {'precision':0.001, plot_dt_steps=10} plot updates plot every 10 x dt-steps 

Simulating (ddt=0.250000) : t=0.000000-->1517.500000 - Steady State reached!
Animation saved to: data/rhoSS1D_temp.mp4


---
## Animated 2D diffusion

### Evolve for finite duration

In [9]:
rho = dfsn.Simulator(n=(12,8),bc=0)
xx,yy = np.meshgrid(np.linspace(-2,2,8),np.linspace(-2,2,12))
f_start = np.exp(-(xx**2+yy**2)/2)
rho.initialize(f=f_start, D=1, B=0)
with dfsn.Animator(simObj=rho, savefile="data/rhoEvo2D.gif") as plotter:
    plotter.start_animation(simulate="evolve", kwargs={'Dt':5,'dt':0.0001, 'plot_realtime_interval':0.1})

Simulating (ddt=0.000100) : t=0.000000-->5.000000 - Evolved!
Animation saved to: data/rhoEvo2Dnew.gif


### Evolve to steady state

In [30]:
rho = dfsn.Simulator(n=(12,8),bc=0)
xx,yy = np.meshgrid(np.linspace(-2,2,8),np.linspace(-2,2,12))
f_start = np.exp(-(xx**2+yy**2)/2)
rho.initialize(f=f_start, D=1, B=0)
with dfsn.Animator(simObj=rho, savefile="data/rhoSS2D.gif", figsize=(9,6)) as plotter:
    plotter.start_animation(simulate="steadyState", kwargs={'precision':0.0001, 'plot_dt_steps':1})

Simulating (ddt=0.125000) : t=0.000000-->9.250000 - Steady State reached!
Animation saved to: data/rhoSS2D.gif


---
## Animated 3D diffusion

### Evolve for finite duration

In [28]:
rho = dfsn.Simulator(n=(5,5,5),bc=0)
xxx,yyy,zzz = np.meshgrid(np.linspace(-2,2,5),np.linspace(-2,2,5),np.linspace(-2,2,5))
f_start = np.exp(-(xxx**2+yyy**2+zzz**2)/2)
rho.initialize(f=f_start, D=1, B=0)
with dfsn.Animator(simObj=rho, savefile="data/rhoEvo3D.gif") as plotter:
    plotter.start_animation(simulate="evolve", kwargs={'Dt':0.5,'dt':0.00001, 'plot_realtime_interval':0.1})

Simulating (ddt=0.000010) : t=0.000000-->0.499990 - Evolved!
Animation saved to: data/rhoEvo3D.gif


### Evolve to steady state

In [29]:
rho = dfsn.Simulator(n=(5,5,5),bc=0)
xxx,yyy,zzz = np.meshgrid(np.linspace(-2,2,5),np.linspace(-2,2,5),np.linspace(-2,2,5))
f_start = np.exp(-(xxx**2+yyy**2+zzz**2)/2)
rho.initialize(f=f_start, D=1, B=0)
with dfsn.Animator(simObj=rho, savefile="data/rhoSS3D.gif") as plotter:
    plotter.start_animation(simulate="steadyState", kwargs={'precision':0.0001, 'plot_dt_steps': 1})

Simulating (ddt=0.083333) : t=0.000000-->2.250000 - Steady State reached!
Animation saved to: data/rhoSS3D.gif


# More help:

In [ ]:
help(dfsn)

## to reload changes in the imported library
from importlib import reload
reload(dfsn)